In [32]:
install.packages("RSQLite")


The downloaded binary packages are in
	/var/folders/wy/f2g87lw121qf8pt2k7_d4ysr0000gn/T//Rtmph3nqp4/downloaded_packages


In [33]:
library(RSQLite)

con <- dbConnect(SQLite(), ":memory:")


In [34]:
library(tidyverse)

In [35]:
# Load CSVs
seoul_bike <- read_csv("seoul_bike_sharing.csv")
cities_weather <- read_csv("cities_weather_forecast.csv")
bike_systems <- read_csv("bike_sharing_systems.csv")
world_cities <- read_csv("world_cities.csv")

# Write to SQLite tables
dbWriteTable(con, "seoul_bike_sharing", seoul_bike)
dbWriteTable(con, "cities_weather_forecast", cities_weather)
dbWriteTable(con, "bike_sharing_systems", bike_systems)
dbWriteTable(con, "world_cities", world_cities)


Rows: 8465 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DATE, SEASONS, HOLIDAY, FUNCTIONING_DAY
dbl (10): RENTED_BIKE_COUNT, HOUR, TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBI...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 160 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): CITY, WEATHER, SEASON
dbl  (8): VISIBILITY, TEMP, TEMP_MIN, TEMP_MAX, PRESSURE, HUMIDITY, WIND_SPE...
dttm (1): FORECAST_DATETIME

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 480 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): COUNTRY, CITY, SYSTEM
dbl (1): BICYCLES

ℹ Use `spec()` t

In [36]:
library(DBI)
dbGetQuery(con, "SELECT COUNT(*) FROM seoul_bike_sharing")


COUNT(*)
<int>
8465


In [37]:
dbGetQuery(con, "
  SELECT COUNT(*) AS non_zero_hours
  FROM seoul_bike_sharing
  WHERE rented_bike_count > 0
")


non_zero_hours
<int>
8465


In [38]:
dbGetQuery(con, "
  SELECT *
  FROM cities_weather_forecast
  WHERE CITY = 'Seoul'
  ORDER BY FORECAST_DATETIME
  LIMIT 1
")



CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,1618574400


In [39]:
dbGetQuery(con, "
  SELECT DISTINCT SEASONS
  FROM seoul_bike_sharing
")


SEASONS
<chr>
Winter
Spring
Summer
Autumn


In [41]:
dbGetQuery(con, "
  SELECT 
    MIN(substr(DATE, 7, 4) || '-' || substr(DATE, 4, 2) || '-' || substr(DATE, 1, 2)) AS first_date,
    MAX(substr(DATE, 7, 4) || '-' || substr(DATE, 4, 2) || '-' || substr(DATE, 1, 2)) AS last_date
  FROM seoul_bike_sharing
")



first_date,last_date
<chr>,<chr>
2017-12-01,2018-11-30


In [42]:
dbGetQuery(con, "
  SELECT DATE, HOUR, RENTED_BIKE_COUNT
  FROM seoul_bike_sharing
  WHERE RENTED_BIKE_COUNT = (
    SELECT MAX(RENTED_BIKE_COUNT) FROM seoul_bike_sharing
  )
")


DATE,HOUR,RENTED_BIKE_COUNT
<chr>,<dbl>,<dbl>
19/06/2018,18,3556


In [43]:
dbGetQuery(con, "
  SELECT SEASONS AS season,
         HOUR AS hour,
         AVG(TEMPERATURE) AS avg_temperature,
         AVG(RENTED_BIKE_COUNT) AS avg_bike_rentals
  FROM seoul_bike_sharing
  GROUP BY SEASONS, HOUR
  ORDER BY avg_bike_rentals DESC
  LIMIT 10
")


season,hour,avg_temperature,avg_bike_rentals
<chr>,<dbl>,<dbl>,<dbl>
Summer,18,29.38791,2135.141
Autumn,18,16.03185,1983.333
Summer,19,28.27378,1889.250
Summer,20,27.06630,1801.924
Summer,21,26.27826,1754.065
Spring,18,15.97222,1689.311
Summer,22,25.69891,1567.870
Autumn,17,17.27778,1562.877
Summer,17,30.07691,1526.293


In [44]:
dbGetQuery(con, "
  SELECT SEASONS AS season,
         AVG(RENTED_BIKE_COUNT) AS avg_bike_count,
         MIN(RENTED_BIKE_COUNT) AS min_bike_count,
         MAX(RENTED_BIKE_COUNT) AS max_bike_count,
         SQRT(AVG(RENTED_BIKE_COUNT * RENTED_BIKE_COUNT) - AVG(RENTED_BIKE_COUNT) * AVG(RENTED_BIKE_COUNT)) AS stddev_bike_count
  FROM seoul_bike_sharing
  GROUP BY SEASONS
")


season,avg_bike_count,min_bike_count,max_bike_count,stddev_bike_count
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Autumn,924.1105,2,3298,617.3885
Spring,746.2542,2,3251,618.5247
Summer,1034.0734,9,3556,690.0884
Winter,225.5412,3,937,150.3374


In [45]:
dbGetQuery(con, "
  SELECT SEASONS AS season,
         AVG(TEMPERATURE) AS avg_temperature,
         AVG(HUMIDITY) AS avg_humidity,
         AVG(WIND_SPEED) AS avg_wind_speed,
         AVG(VISIBILITY) AS avg_visibility,
         AVG(DEW_POINT_TEMPERATURE) AS avg_dew_point_temperature,
         AVG(SOLAR_RADIATION) AS avg_solar_radiation,
         AVG(RAINFALL) AS avg_rainfall,
         AVG(SNOWFALL) AS avg_snowfall,
         AVG(RENTED_BIKE_COUNT) AS avg_bike_count
  FROM seoul_bike_sharing
  GROUP BY SEASONS
  ORDER BY avg_bike_count DESC
")


season,avg_temperature,avg_humidity,avg_wind_speed,avg_visibility,avg_dew_point_temperature,avg_solar_radiation,avg_rainfall,avg_snowfall,avg_bike_count
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Summer,26.587711,64.98143,1.609420,1501.745,18.750136,0.7612545,0.25348732,0.00000000,1034.0734
Autumn,13.821580,59.04491,1.492101,1558.174,5.150594,0.5227827,0.11765617,0.06350026,924.1105
Spring,13.021685,58.75833,1.857778,1240.912,4.091389,0.6803009,0.18694444,0.00000000,746.2542
Winter,-2.540463,49.74491,1.922685,1445.987,-12.416667,0.2981806,0.03282407,0.24750000,225.5412


In [48]:
dbGetQuery(con, "
  SELECT w.CITY,
         w.COUNTRY,
         w.LAT,
         w.LNG,
         w.POPULATION,
         b.BICYCLES AS total_bikes
  FROM world_cities w, bike_sharing_systems b
  WHERE w.CITY = 'Seoul'
    AND b.CITY = 'Seoul'
")


CITY,COUNTRY,LAT,LNG,POPULATION,total_bikes
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Seoul,"Korea, South",37.5833,127,21794000,20000


In [49]:
dbGetQuery(con, "
  SELECT w.CITY,
         w.COUNTRY,
         w.LAT,
         w.LNG,
         w.POPULATION,
         b.BICYCLES AS total_bikes
  FROM world_cities w
  JOIN bike_sharing_systems b ON w.CITY = b.CITY
  WHERE b.BICYCLES BETWEEN 15000 AND 20000
")


CITY,COUNTRY,LAT,LNG,POPULATION,total_bikes
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Beijing,China,39.9050,116.3914,19433000,16000
Ningbo,China,29.8750,121.5492,7639000,15000
Shanghai,China,31.1667,121.4667,22120000,19165
Weifang,China,36.7167,119.1000,9373000,20000
Zhuzhou,China,27.8407,113.1469,3855609,20000
Seoul,"Korea, South",37.5833,127.0000,21794000,20000
